# CLARISSA Code Examples

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gl/wolfram_laube%2Fblauweiss_llc%2Firena/main?labpath=docs%2Ftutorials%2Freservoir-basics-code.ipynb)

**💡 Click the Binder badge above to run this notebook interactively!**

Executable Python code from the [Reservoir Basics Tutorial](reservoir-basics.md).

## Pore Volume Calculation

In [ ]:
# Reservoir parameters (SPE9 model)
nx, ny, nz = 24, 25, 15  # Grid dimensions
dx, dy, dz = 300, 300, 50  # Cell sizes in feet
porosity = 0.087  # Average porosity
ntg = 1.0  # Net-to-gross ratio

# Calculate volumes
bulk_volume = nx * ny * nz * dx * dy * dz  # ft³
pore_volume = bulk_volume * porosity * ntg
pore_volume_bbl = pore_volume / 5.615  # Convert to barrels

print(f'Grid: {nx} x {ny} x {nz} = {nx*ny*nz:,} cells')
print(f'Pore volume: {pore_volume_bbl/1e6:.2f} MMbbl')

## Material Balance - OOIP

In [ ]:
def calculate_ooip(np_cum, bo, boi, delta_p, ce):
    """Calculate OOIP using simplified material balance."""
    return (np_cum * bo) / (boi * ce * delta_p)

# Example
N = calculate_ooip(
    np_cum=1_500_000,  # STB produced
    bo=1.25,
    boi=1.30,
    delta_p=500,  # psi
    ce=15e-6  # 1/psi
)

print(f'Estimated OOIP: {N/1e6:.1f} MMSTB')

## Recovery Factor Estimation

In [ ]:
# Typical recovery factors by drive mechanism
drive_mechanisms = {
    'Solution gas drive': (0.05, 0.30),
    'Gas cap drive': (0.20, 0.40),
    'Water drive': (0.35, 0.75),
    'Gravity drainage': (0.50, 0.70),
}

ooip_mmstb = 150  # Million STB

print(f'OOIP: {ooip_mmstb} MMSTB\n')
print(f'{"Drive Mechanism":<25} {"RF Range":>15} {"Recoverable (MMSTB)":>20}')
print('-' * 62)

for mechanism, (rf_low, rf_high) in drive_mechanisms.items():
    rec_low = ooip_mmstb * rf_low
    rec_high = ooip_mmstb * rf_high
    print(f'{mechanism:<25} {rf_low*100:>5.0f}% - {rf_high*100:<5.0f}% {rec_low:>8.1f} - {rec_high:<8.1f}')